# Merge Events from Assocation from PyOcto ver. 3 after cross correlation

In [1]:
import os
import pandas as pd
import numpy as np
from obspy.geodetics import locations2degrees, degrees2kilometers 
from obspy.geodetics.base import gps2dist_azimuth
from tqdm import tqdm
import datetime
from obspy.clients.fdsn import Client





## Make the assignment data frame

In [2]:
region = 'all_regions'

picks = pd.read_csv(f'/wd1/hbito_data/data/datasets_{region}/Cascadia_relocated_catalog_picks_ver_3.csv')
picks

,Pick Time (UTC),Station Name,Phase Type,Residual (s),Event ID,Pick ID
0,2010-01-01T00:15:27.180000Z,PCMD.UW,0,0.049,0,0
1,2010-01-01T00:15:37.840400Z,RVW.UW,0,1.264,0,1
2,2010-01-01T00:15:33.280000Z,PCMD.UW,1,-0.243,0,2
3,2010-01-01T00:15:42.002000Z,GNW.UW,1,2.402,0,3
4,2010-01-01T00:15:43.618400Z,B013.PB,1,-0.651,0,4
...,...,...,...,...,...,...
1004330,2015-06-23T23:18:40.885798Z,J11D.7D,1,0.044,63886,1004330
1004331,2015-06-23T23:18:48.573898Z,G35D.7D,1,0.358,63886,1004331
1004332,2015-06-23T23:18:50.458298Z,J19D.7D,1,0.300,63886,1004332
1004333,2015-06-23T23:18:56.689277Z,J10D.7D,1,0.432,63886,1004333


In [3]:
events = pd.read_csv(f'/wd1/hbito_data/data/datasets_all_regions/Cascadia_updated_catalog_ver3.csv')
events

,Latitude,Longitude,Event ID,Depth (km),Detection Value,Origin Time (UTC),RMS Residual (s),Num. P,Num. S
0,47.13396,-122.09098,0,60.1470,0.680,2010-01-01T00:15:16.204000Z,1.081,2,5
1,48.17742,-121.83289,1,6.1630,0.840,2010-01-01T00:16:49.343000Z,0.985,25,30
2,47.85353,-122.12435,2,18.2960,0.741,2010-01-01T07:18:03.837000Z,0.784,10,18
3,47.97665,-122.90617,3,20.9100,0.756,2010-01-01T08:51:56.196000Z,0.465,10,10
4,45.86019,-122.19002,4,8.4035,0.850,2010-01-01T16:12:43.926000Z,0.657,20,19
...,...,...,...,...,...,...,...,...,...
63882,40.59877,-124.40572,63882,20.4825,0.654,2015-06-23T22:11:13.828000Z,1.015,9,14
63883,40.62611,-127.11445,63883,18.5140,0.840,2015-06-23T22:45:50.498000Z,0.744,1,11
63884,49.76714,-124.52950,63884,3.6850,0.801,2015-06-23T23:03:50.163000Z,0.894,7,6
63885,40.66547,-125.33902,63885,3.0525,0.633,2015-06-23T23:12:22.778000Z,0.982,3,5


In [4]:
region = 'all_regions'

picks = pd.read_csv(f'/wd1/hbito_data/data/datasets_{region}/Cascadia_relocated_catalog_picks_ver_3.csv')
events = events.rename(columns={' Event ID ':'idx',' Origin Time (UTC)':'datetime','Latitude':'latitude','Longitude':'longitude',' Depth (km)':'depth',' Num. P':'Num. P', ' Num. S':'Num. S',' RMS Residual (s)':'RMS Residual (s)', ' Detection Value':'Detection Value'})
picks = picks.rename(columns={' Event ID':'idx',' Station Name':'station', ' Phase Type':'phase',' Pick ID ':'arid','Pick Time (UTC)':'time_pick',' Residual (s)':'timeres'})
events['datetime'] = pd.to_datetime(events['datetime'], utc=True)
picks['station'] = picks['station'].str.strip()
# Flip the order of the station name and network name
picks['station'] = picks['station'].apply(lambda x: '.'.join(x.split('.')[::-1]))
picks['phase'] = picks['phase'].replace({0: 'P', 1: 'S'})
picks = picks.merge(events, on='idx', how='left')
# mycatalog = picks
mycatalog = events
mycatalog

,latitude,longitude,idx,depth,Detection Value,datetime,RMS Residual (s),Num. P,Num. S
0,47.13396,-122.09098,0,60.1470,0.680,2010-01-01 00:15:16.204000+00:00,1.081,2,5
1,48.17742,-121.83289,1,6.1630,0.840,2010-01-01 00:16:49.343000+00:00,0.985,25,30
2,47.85353,-122.12435,2,18.2960,0.741,2010-01-01 07:18:03.837000+00:00,0.784,10,18
3,47.97665,-122.90617,3,20.9100,0.756,2010-01-01 08:51:56.196000+00:00,0.465,10,10
4,45.86019,-122.19002,4,8.4035,0.850,2010-01-01 16:12:43.926000+00:00,0.657,20,19
...,...,...,...,...,...,...,...,...,...
63882,40.59877,-124.40572,63882,20.4825,0.654,2015-06-23 22:11:13.828000+00:00,1.015,9,14
63883,40.62611,-127.11445,63883,18.5140,0.840,2015-06-23 22:45:50.498000+00:00,0.744,1,11
63884,49.76714,-124.52950,63884,3.6850,0.801,2015-06-23 23:03:50.163000+00:00,0.894,7,6
63885,40.66547,-125.33902,63885,3.0525,0.633,2015-06-23 23:12:22.778000+00:00,0.982,3,5


In [5]:
# picks = pd.read_csv(f'/wd1/hbito_data/data/datasets_all_regions/Cascadia_updated_catalog_picks_ver_3.csv')
# picks

In [6]:
picks['picks'] = picks['Num. P'] + picks['Num. S']
picks

,time_pick,station,phase,timeres,idx,arid,latitude,longitude,depth,Detection Value,datetime,RMS Residual (s),Num. P,Num. S,picks
0,2010-01-01T00:15:27.180000Z,UW.PCMD,P,0.049,0,0,47.13396,-122.09098,60.1470,0.680,2010-01-01 00:15:16.204000+00:00,1.081,2,5,7
1,2010-01-01T00:15:37.840400Z,UW.RVW,P,1.264,0,1,47.13396,-122.09098,60.1470,0.680,2010-01-01 00:15:16.204000+00:00,1.081,2,5,7
2,2010-01-01T00:15:33.280000Z,UW.PCMD,S,-0.243,0,2,47.13396,-122.09098,60.1470,0.680,2010-01-01 00:15:16.204000+00:00,1.081,2,5,7
3,2010-01-01T00:15:42.002000Z,UW.GNW,S,2.402,0,3,47.13396,-122.09098,60.1470,0.680,2010-01-01 00:15:16.204000+00:00,1.081,2,5,7
4,2010-01-01T00:15:43.618400Z,PB.B013,S,-0.651,0,4,47.13396,-122.09098,60.1470,0.680,2010-01-01 00:15:16.204000+00:00,1.081,2,5,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004330,2015-06-23T23:18:40.885798Z,7D.J11D,S,0.044,63886,1004330,43.33308,-127.31240,6.0615,0.694,2015-06-23 23:18:19.457000+00:00,0.447,4,5,9
1004331,2015-06-23T23:18:48.573898Z,7D.G35D,S,0.358,63886,1004331,43.33308,-127.31240,6.0615,0.694,2015-06-23 23:18:19.457000+00:00,0.447,4,5,9
1004332,2015-06-23T23:18:50.458298Z,7D.J19D,S,0.300,63886,1004332,43.33308,-127.31240,6.0615,0.694,2015-06-23 23:18:19.457000+00:00,0.447,4,5,9
1004333,2015-06-23T23:18:56.689277Z,7D.J10D,S,0.432,63886,1004333,43.33308,-127.31240,6.0615,0.694,2015-06-23 23:18:19.457000+00:00,0.447,4,5,9


In [7]:
picks.to_csv('/wd1/hbito_data/data/datasets_all_regions/Cascadia_updated_catalog_picks_assignment_ver_3.csv')

In [8]:
df_assignments_concat = pd.read_csv('/wd1/hbito_data/data/datasets_all_regions/Cascadia_updated_catalog_picks_assignment_ver_3.csv')
df_assignments_concat = df_assignments_concat.rename(columns={'datetime': 'time'})
df_assignments_concat

,Unnamed: 0,time_pick,station,phase,timeres,idx,arid,latitude,longitude,depth,Detection Value,time,RMS Residual (s),Num. P,Num. S,picks
0,0,2010-01-01T00:15:27.180000Z,UW.PCMD,P,0.049,0,0,47.13396,-122.09098,60.1470,0.680,2010-01-01 00:15:16.204000+00:00,1.081,2,5,7
1,1,2010-01-01T00:15:37.840400Z,UW.RVW,P,1.264,0,1,47.13396,-122.09098,60.1470,0.680,2010-01-01 00:15:16.204000+00:00,1.081,2,5,7
2,2,2010-01-01T00:15:33.280000Z,UW.PCMD,S,-0.243,0,2,47.13396,-122.09098,60.1470,0.680,2010-01-01 00:15:16.204000+00:00,1.081,2,5,7
3,3,2010-01-01T00:15:42.002000Z,UW.GNW,S,2.402,0,3,47.13396,-122.09098,60.1470,0.680,2010-01-01 00:15:16.204000+00:00,1.081,2,5,7
4,4,2010-01-01T00:15:43.618400Z,PB.B013,S,-0.651,0,4,47.13396,-122.09098,60.1470,0.680,2010-01-01 00:15:16.204000+00:00,1.081,2,5,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004330,1004330,2015-06-23T23:18:40.885798Z,7D.J11D,S,0.044,63886,1004330,43.33308,-127.31240,6.0615,0.694,2015-06-23 23:18:19.457000+00:00,0.447,4,5,9
1004331,1004331,2015-06-23T23:18:48.573898Z,7D.G35D,S,0.358,63886,1004331,43.33308,-127.31240,6.0615,0.694,2015-06-23 23:18:19.457000+00:00,0.447,4,5,9
1004332,1004332,2015-06-23T23:18:50.458298Z,7D.J19D,S,0.300,63886,1004332,43.33308,-127.31240,6.0615,0.694,2015-06-23 23:18:19.457000+00:00,0.447,4,5,9
1004333,1004333,2015-06-23T23:18:56.689277Z,7D.J10D,S,0.432,63886,1004333,43.33308,-127.31240,6.0615,0.694,2015-06-23 23:18:19.457000+00:00,0.447,4,5,9


In [9]:
# events = pd.read_csv('../data/datasets_all_regions/Cascadia_relocated_catalog_ver_3.csv')
# picks = pd.read_csv('../data/datasets_all_regions/Cascadia_relocated_catalog_picks_ver_3.csv')

In [10]:
# picks

In [11]:
# events

## Make a data frame for station information

In [12]:
# # Load all the pick assignments 
# mycatalog_picks = pd.read_csv('../data/datasets_all_regions/Cascadia_relocated_catalog_picks_ver_3.csv')
# # Initialize lists to store station information
# station_lats = []
# station_lons = []
# station_elevs = []
# station_names = []

# client = Client("IRIS")
# # Loop through each station in mycatalog_picks
# stas = mycatalog_picks[' Station Name'].unique()

# for sta in tqdm(stas, total=len(stas)):
#     network = sta.split('.')[1]
#     station = sta.split('.')[0]
#     try:
#         # Fetch station metadata
#         inventory = client.get_stations(network=network, station=station, level="station")
#         # Extract latitude, longitude, and elevation
#         lat = inventory[0][0].latitude
#         lon = inventory[0][0].longitude
#         elev = inventory[0][0].elevation
#     except:
#         # If station metadata is not found, set values to None
#         lat, lon, elev = None, None, None
    
#     # Append the information to the lists
#     station_lats.append(lat)
#     station_lons.append(lon)
#     station_elevs.append(elev)
#     station_names.append(network.strip() + '.' + station.strip())

# # Create a DataFrame with the station information
# station_info = pd.DataFrame({
#     'station': station_names,
#     'latitude': station_lats,
#     'longitude': station_lons,
#     'elevation': station_elevs
# })

# station_info.to_csv('../data/datasets_all_regions/station_info_Cascadia_catalog_ver_3.csv')

In [13]:
station_info = pd.read_csv('/wd1/hbito_data/data/datasets_all_regions/station_info_Cascadia_catalog_ver_3.csv', index_col=0)
station_info = station_info.rename(columns={'latitude':'slatitude','longitude':'slongitude','elevation':'selevation'})
station_info

,station,slatitude,slongitude,selevation
0,UW.PCMD,46.888962,-122.301483,239.0
1,UW.RVW,46.149750,-122.742996,504.0
2,UW.GNW,47.564130,-122.824980,220.0
3,PB.B013,47.813000,-122.910797,75.3
4,PB.B943,47.813202,-122.911301,84.2
...,...,...,...,...
435,OO.HYSB1,44.509791,-125.405258,-2908.0
436,7D.M13D,43.597301,-125.044601,-990.0
437,OO.HYS14,44.569157,-125.147900,-775.0
438,C8.SPLB,49.892100,-125.652600,276.0


In [14]:
df_assignments_concat = df_assignments_concat.merge(station_info, on='station', how='left')
df_assignments_concat

,Unnamed: 0,time_pick,station,phase,timeres,idx,arid,latitude,longitude,depth,Detection Value,time,RMS Residual (s),Num. P,Num. S,picks,slatitude,slongitude,selevation
0,0,2010-01-01T00:15:27.180000Z,UW.PCMD,P,0.049,0,0,47.13396,-122.09098,60.1470,0.680,2010-01-01 00:15:16.204000+00:00,1.081,2,5,7,46.888962,-122.301483,239.0
1,1,2010-01-01T00:15:37.840400Z,UW.RVW,P,1.264,0,1,47.13396,-122.09098,60.1470,0.680,2010-01-01 00:15:16.204000+00:00,1.081,2,5,7,46.149750,-122.742996,504.0
2,2,2010-01-01T00:15:33.280000Z,UW.PCMD,S,-0.243,0,2,47.13396,-122.09098,60.1470,0.680,2010-01-01 00:15:16.204000+00:00,1.081,2,5,7,46.888962,-122.301483,239.0
3,3,2010-01-01T00:15:42.002000Z,UW.GNW,S,2.402,0,3,47.13396,-122.09098,60.1470,0.680,2010-01-01 00:15:16.204000+00:00,1.081,2,5,7,47.564130,-122.824980,220.0
4,4,2010-01-01T00:15:43.618400Z,PB.B013,S,-0.651,0,4,47.13396,-122.09098,60.1470,0.680,2010-01-01 00:15:16.204000+00:00,1.081,2,5,7,47.813000,-122.910797,75.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004330,1004330,2015-06-23T23:18:40.885798Z,7D.J11D,S,0.044,63886,1004330,43.33308,-127.31240,6.0615,0.694,2015-06-23 23:18:19.457000+00:00,0.447,4,5,9,43.541599,-126.368599,-3000.8
1004331,1004331,2015-06-23T23:18:48.573898Z,7D.G35D,S,0.358,63886,1004331,43.33308,-127.31240,6.0615,0.694,2015-06-23 23:18:19.457000+00:00,0.447,4,5,9,42.555698,-126.399002,-2822.6
1004332,1004332,2015-06-23T23:18:50.458298Z,7D.J19D,S,0.300,63886,1004332,43.33308,-127.31240,6.0615,0.694,2015-06-23 23:18:19.457000+00:00,0.447,4,5,9,44.179001,-126.271202,-2955.4
1004333,1004333,2015-06-23T23:18:56.689277Z,7D.J10D,S,0.432,63886,1004333,43.33308,-127.31240,6.0615,0.694,2015-06-23 23:18:19.457000+00:00,0.447,4,5,9,43.348499,-125.545097,-3085.0


In [15]:
# Save the assignment data frame
df_assignments_concat.to_csv('/wd1/hbito_data/data/datasets_all_regions/Cascadia_updated_catalog_picks_assignment_ver_3.csv', index=False)

## Make the arrival table

In [16]:
df_arrival  = df_assignments_concat[['station', 'time_pick', 'arid','phase','Detection Value']]
df_arrival = df_arrival.rename(columns={'station':'sta','time_pick': 'time','phase':'iphase','Detection Value':'prob'})
df_arrival['time'] = pd.to_datetime(df_arrival["time"]).apply(lambda x: x.timestamp())
df_arrival 

,sta,time,arid,iphase,prob
0,UW.PCMD,1.262305e+09,0,P,0.680
1,UW.RVW,1.262305e+09,1,P,0.680
2,UW.PCMD,1.262305e+09,2,S,0.680
3,UW.GNW,1.262305e+09,3,S,0.680
4,PB.B013,1.262305e+09,4,S,0.680
...,...,...,...,...,...
1004330,7D.J11D,1.435102e+09,1004330,S,0.694
1004331,7D.G35D,1.435102e+09,1004331,S,0.694
1004332,7D.J19D,1.435102e+09,1004332,S,0.694
1004333,7D.J10D,1.435102e+09,1004333,S,0.694


In [17]:
# import numpy as np

# # Replace NaN with np.nan (which is already float64) and ensure the column is of type float64
# df['column_name'] = df['column_name'].astype('float64')

In [18]:
df_arrival.to_csv('/wd1/hbito_data/data/datasets_all_regions/arrival_2010_2015_reloc_cog_ver3_cc.csv')

In [19]:
# df_arrival[df_arrival['chan'] == np.nan]

In [20]:
df_arrival = pd.read_csv('/wd1/hbito_data/data/datasets_all_regions/arrival_2010_2015_reloc_cog_ver3_cc.csv',index_col=0)
df_arrival.head()

,sta,time,arid,iphase,prob
0,UW.PCMD,1.262305e+09,0,P,0.68
1,UW.RVW,1.262305e+09,1,P,0.68
2,UW.PCMD,1.262305e+09,2,S,0.68
3,UW.GNW,1.262305e+09,3,S,0.68
4,PB.B013,1.262305e+09,4,S,0.68


## Make the assoc table

In [21]:
# Assignn unique origin indices starting from 0 to each event
df_assignments_concat_orid = df_assignments_concat.rename(columns={'idx': 'orid'})



In [22]:
# Retrieve and rename the columns needed for the association table 
df_assoc = df_assignments_concat_orid[['arid', 'orid', 'station','phase','Detection Value','timeres','slatitude','slongitude','selevation']]
df_assoc = df_assoc.rename(columns={'station':'sta','Detection Value':'prob'})
df_assoc

,arid,orid,sta,phase,prob,timeres,slatitude,slongitude,selevation
0,0,0,UW.PCMD,P,0.680,0.049,46.888962,-122.301483,239.0
1,1,0,UW.RVW,P,0.680,1.264,46.149750,-122.742996,504.0
2,2,0,UW.PCMD,S,0.680,-0.243,46.888962,-122.301483,239.0
3,3,0,UW.GNW,S,0.680,2.402,47.564130,-122.824980,220.0
4,4,0,PB.B013,S,0.680,-0.651,47.813000,-122.910797,75.3
...,...,...,...,...,...,...,...,...,...
1004330,1004330,63886,7D.J11D,S,0.694,0.044,43.541599,-126.368599,-3000.8
1004331,1004331,63886,7D.G35D,S,0.694,0.358,42.555698,-126.399002,-2822.6
1004332,1004332,63886,7D.J19D,S,0.694,0.300,44.179001,-126.271202,-2955.4
1004333,1004333,63886,7D.J10D,S,0.694,0.432,43.348499,-125.545097,-3085.0


## Make the origin table
Assuming 'time' is unique.

In [23]:
df_assignments_concat

,Unnamed: 0,time_pick,station,phase,timeres,idx,arid,latitude,longitude,depth,Detection Value,time,RMS Residual (s),Num. P,Num. S,picks,slatitude,slongitude,selevation
0,0,2010-01-01T00:15:27.180000Z,UW.PCMD,P,0.049,0,0,47.13396,-122.09098,60.1470,0.680,2010-01-01 00:15:16.204000+00:00,1.081,2,5,7,46.888962,-122.301483,239.0
1,1,2010-01-01T00:15:37.840400Z,UW.RVW,P,1.264,0,1,47.13396,-122.09098,60.1470,0.680,2010-01-01 00:15:16.204000+00:00,1.081,2,5,7,46.149750,-122.742996,504.0
2,2,2010-01-01T00:15:33.280000Z,UW.PCMD,S,-0.243,0,2,47.13396,-122.09098,60.1470,0.680,2010-01-01 00:15:16.204000+00:00,1.081,2,5,7,46.888962,-122.301483,239.0
3,3,2010-01-01T00:15:42.002000Z,UW.GNW,S,2.402,0,3,47.13396,-122.09098,60.1470,0.680,2010-01-01 00:15:16.204000+00:00,1.081,2,5,7,47.564130,-122.824980,220.0
4,4,2010-01-01T00:15:43.618400Z,PB.B013,S,-0.651,0,4,47.13396,-122.09098,60.1470,0.680,2010-01-01 00:15:16.204000+00:00,1.081,2,5,7,47.813000,-122.910797,75.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004330,1004330,2015-06-23T23:18:40.885798Z,7D.J11D,S,0.044,63886,1004330,43.33308,-127.31240,6.0615,0.694,2015-06-23 23:18:19.457000+00:00,0.447,4,5,9,43.541599,-126.368599,-3000.8
1004331,1004331,2015-06-23T23:18:48.573898Z,7D.G35D,S,0.358,63886,1004331,43.33308,-127.31240,6.0615,0.694,2015-06-23 23:18:19.457000+00:00,0.447,4,5,9,42.555698,-126.399002,-2822.6
1004332,1004332,2015-06-23T23:18:50.458298Z,7D.J19D,S,0.300,63886,1004332,43.33308,-127.31240,6.0615,0.694,2015-06-23 23:18:19.457000+00:00,0.447,4,5,9,44.179001,-126.271202,-2955.4
1004333,1004333,2015-06-23T23:18:56.689277Z,7D.J10D,S,0.432,63886,1004333,43.33308,-127.31240,6.0615,0.694,2015-06-23 23:18:19.457000+00:00,0.447,4,5,9,43.348499,-125.545097,-3085.0


In [24]:
df_origin = df_assignments_concat_orid[['latitude','longitude','depth','time','orid','picks','Num. P','Num. S', 'RMS Residual (s)']]
df_origin = df_origin.rename(columns={'latitude':'lat','longitude':'lon','picks':'nass','Num. P':'p_picks','Num. S':'s_picks','RMS Residual (s)':'rms'})
df_origin['time'] = pd.to_datetime(df_origin["time"]).apply(lambda x: x.timestamp())
df_origin.drop_duplicates(subset=['orid'], inplace=True)

In [25]:
# Iterate through each origin ID in the ORIGIN table
for orid in tqdm(df_origin['orid'],total=len(df_origin['orid'])):
    # Filter the association table for the current origin ID
    _df_assoc = df_assoc[df_assoc['orid'] == orid]
    
    esaz_values = []
    timeres_values = []
    s_picks_count = 0

    # Get origin latitude and longitude
    origin_lat = df_origin.loc[df_origin['orid'] == orid, 'lat'].iloc[0]
    origin_lon = df_origin.loc[df_origin['orid'] == orid, 'lon'].iloc[0]

    for _, row in _df_assoc.iterrows():
        # Get station latitude and longitude
        station_lat = row['slatitude']
        station_lon = row['slongitude']

        # Calculate delta, esaz, and seaz
        delta_m, azimuth, back_azimuth = gps2dist_azimuth(origin_lat, origin_lon, station_lat, station_lon)
        delta_km = delta_m * 1e-3  # Convert meters to degrees
        esaz = (back_azimuth + 180) % 360  # Ensure esaz is in [0, 360)
        seaz = back_azimuth

        # Append esaz for gap calculation
        esaz_values.append(esaz)

        # Count S-picks
        if row['phase'] == 'S':
            s_picks_count += 1

        # Update ASSOC table with calculated values
        df_assoc.loc[row.name, 'delta'] = delta_km
        df_assoc.loc[row.name, 'esaz'] = esaz
        df_assoc.loc[row.name, 'seaz'] = seaz

    # Calculate gap
    esaz_values.sort()
    gaps = [(esaz_values[i] - esaz_values[i - 1]) % 360 for i in range(len(esaz_values))]
    gap = max(gaps) if gaps else None

    # Calculate RMS of timeres

    # Update ORIGIN table with calculated values
    df_origin.loc[df_origin['orid'] == orid, 'nsphz'] = int(s_picks_count)
    df_origin.loc[df_origin['orid'] == orid, 'gap'] = gap

# Add algorithm and lddate fields
df_origin['algorithm'] = 'genie'

100%|██████████| 63887/63887 [13:55<00:00, 76.48it/s] 


In [27]:
df_origin

,lat,lon,depth,time,orid,nass,p_picks,s_picks,rms,nsphz,gap,algorithm
0,47.13396,-122.09098,60.1470,1.262305e+09,0,7,2,5,1.081,5.0,243.895048,genie
7,48.17742,-121.83289,6.1630,1.262305e+09,1,55,25,30,0.985,30.0,197.622556,genie
62,47.85353,-122.12435,18.2960,1.262330e+09,2,28,10,18,0.784,18.0,179.587425,genie
90,47.97665,-122.90617,20.9100,1.262336e+09,3,20,10,10,0.465,10.0,153.822127,genie
110,45.86019,-122.19002,8.4035,1.262362e+09,4,39,20,19,0.657,19.0,238.225383,genie
...,...,...,...,...,...,...,...,...,...,...,...,...
1004270,40.59877,-124.40572,20.4825,1.435097e+09,63882,23,9,14,1.015,14.0,77.202555,genie
1004293,40.62611,-127.11445,18.5140,1.435100e+09,63883,12,1,11,0.744,11.0,221.215338,genie
1004305,49.76714,-124.52950,3.6850,1.435101e+09,63884,13,7,6,0.894,6.0,205.239215,genie
1004318,40.66547,-125.33902,3.0525,1.435101e+09,63885,8,3,5,0.982,5.0,141.141631,genie


In [28]:
df_assoc

,arid,orid,sta,phase,prob,timeres,slatitude,slongitude,selevation,delta,esaz,seaz
0,0,0,UW.PCMD,P,0.680,0.049,46.888962,-122.301483,239.0,31.591871,210.365169,30.365169
1,1,0,UW.RVW,P,0.680,1.264,46.149750,-122.742996,504.0,120.257941,204.289356,24.289356
2,2,0,UW.PCMD,S,0.680,-0.243,46.888962,-122.301483,239.0,31.591871,210.365169,30.365169
3,3,0,UW.GNW,S,0.680,2.402,47.564130,-122.824980,220.0,73.232761,310.501821,130.501821
4,4,0,PB.B013,S,0.680,-0.651,47.813000,-122.910797,75.3,97.562718,320.394308,140.394308
...,...,...,...,...,...,...,...,...,...,...,...,...
1004330,1004330,63886,7D.J11D,S,0.694,0.044,43.541599,-126.368599,-3000.8,79.844583,73.458151,253.458151
1004331,1004331,63886,7D.G35D,S,0.694,0.358,42.555698,-126.399002,-2822.6,114.083213,139.509592,319.509592
1004332,1004332,63886,7D.J19D,S,0.694,0.300,44.179001,-126.271202,-2955.4,125.953761,42.098981,222.098981
1004333,1004333,63886,7D.J10D,S,0.694,0.432,43.348499,-125.545097,-3085.0,143.316176,89.921709,269.921709


In [29]:
df_assoc.to_csv('/wd1/hbito_data/data/datasets_all_regions/assoc_2010_2015_reloc_cog_ver3_cc.csv')

In [30]:
df_origin.to_csv('/wd1/hbito_data/data/datasets_all_regions/origin_2010_2015_reloc_cog_ver3_cc.csv')

### Make a dataframe that has all information from the arrival, association, and origin tables

In [31]:
# Read the CSV file for arrivals
csv_path_arrival = '/wd1/hbito_data/data/datasets_all_regions/arrival_2010_2015_reloc_cog_ver3_cc.csv'
# Expand the home directory
expanded_path = os.path.expanduser(csv_path_arrival)

# Check if the file exists
print(f"Checking if file exists at path: {expanded_path}")
if os.path.exists(expanded_path):
    print(f"File exists. Size: {os.path.getsize(expanded_path)} bytes")
    # Try to read the file
    try:
        arrival_df = pd.read_csv(expanded_path)
        print("Successfully read the file!")
        print(arrival_df.head())  # Display the first few rows of the DataFrame
    except Exception as e:
        print(f"Error reading the file: {e}")
else:
    print("File does not exist at the specified path!")
    # Look for similar files in the directory
    dir_path = os.path.dirname(expanded_path)
    print(f"Files in {dir_path}:")
    similar_files = [f for f in os.listdir(dir_path) if 'arrival' in f.lower()]
    for file in similar_files:
        print(f" - {file}")

arrival_df = arrival_df.rename(columns={'time': 'pick_time'})

Checking if file exists at path: /wd1/hbito_data/data/datasets_all_regions/arrival_2010_2015_reloc_cog_ver3_cc.csv
File exists. Size: 45602750 bytes
Successfully read the file!
   Unnamed: 0      sta          time  arid iphase  prob
0           0  UW.PCMD  1.262305e+09     0      P  0.68
1           1   UW.RVW  1.262305e+09     1      P  0.68
2           2  UW.PCMD  1.262305e+09     2      S  0.68
3           3   UW.GNW  1.262305e+09     3      S  0.68
4           4  PB.B013  1.262305e+09     4      S  0.68


In [32]:
# Read the CSV file for arrivals
csv_path_arrival = '/wd1/hbito_data/data/datasets_all_regions/assoc_2010_2015_reloc_cog_ver3_cc.csv'
# Expand the home directory
expanded_path = os.path.expanduser(csv_path_arrival)

# Check if the file exists
print(f"Checking if file exists at path: {expanded_path}")
if os.path.exists(expanded_path):
    print(f"File exists. Size: {os.path.getsize(expanded_path)} bytes")
    # Try to read the file
    try:
        assoc_df = pd.read_csv(expanded_path)
        print("Successfully read the file!")
        print(assoc_df.head())  # Display the first few rows of the DataFrame
    except Exception as e:
        print(f"Error reading the file: {e}")
else:
    print("File does not exist at the specified path!")
    # Look for similar files in the directory
    dir_path = os.path.dirname(expanded_path)
    print(f"Files in {dir_path}:")
    similar_files = [f for f in os.listdir(dir_path) if 'assoc' in f.lower()]
    for file in similar_files:
        print(f" - {file}")

Checking if file exists at path: /wd1/hbito_data/data/datasets_all_regions/assoc_2010_2015_reloc_cog_ver3_cc.csv
File exists. Size: 125036939 bytes
Successfully read the file!
   Unnamed: 0  arid  orid      sta phase  prob  timeres  slatitude  \
0           0     0     0  UW.PCMD     P  0.68    0.049  46.888962   
1           1     1     0   UW.RVW     P  0.68    1.264  46.149750   
2           2     2     0  UW.PCMD     S  0.68   -0.243  46.888962   
3           3     3     0   UW.GNW     S  0.68    2.402  47.564130   
4           4     4     0  PB.B013     S  0.68   -0.651  47.813000   

   slongitude  selevation       delta        esaz        seaz  
0 -122.301483       239.0   31.591871  210.365169   30.365169  
1 -122.742996       504.0  120.257941  204.289356   24.289356  
2 -122.301483       239.0   31.591871  210.365169   30.365169  
3 -122.824980       220.0   73.232761  310.501821  130.501821  
4 -122.910797        75.3   97.562718  320.394308  140.394308  


In [33]:
# Read the CSV file for origins
csv_path_origin = '/wd1/hbito_data/data/datasets_all_regions/origin_2010_2015_reloc_cog_ver3_cc.csv'
# Expand the home directory
expanded_path = os.path.expanduser(csv_path_origin)

# Check if the file exists
print(f"Checking if file exists at path: {expanded_path}")
if os.path.exists(expanded_path):
    print(f"File exists. Size: {os.path.getsize(expanded_path)} bytes")
    # Try to read the file
    try:
        origin_df = pd.read_csv(expanded_path)
        print("Successfully read the file!")
        print(origin_df.head())  # Display the first few rows of the DataFrame
    except Exception as e:
        print(f"Error reading the file: {e}")
else:
    print("File does not exist at the specified path!")
    # Look for similar files in the directory
    dir_path = os.path.dirname(expanded_path)
    print(f"Files in {dir_path}:")
    similar_files = [f for f in os.listdir(dir_path) if 'origin' in f.lower()]
    for file in similar_files:
        print(f" - {file}")
origin_df = origin_df.rename(columns={'time': 'otime'})

Checking if file exists at path: /wd1/hbito_data/data/datasets_all_regions/origin_2010_2015_reloc_cog_ver3_cc.csv
File exists. Size: 6163942 bytes
Successfully read the file!
   Unnamed: 0       lat        lon    depth          time  orid  nass  \
0           0  47.13396 -122.09098  60.1470  1.262305e+09     0     7   
1           7  48.17742 -121.83289   6.1630  1.262305e+09     1    55   
2          62  47.85353 -122.12435  18.2960  1.262330e+09     2    28   
3          90  47.97665 -122.90617  20.9100  1.262336e+09     3    20   
4         110  45.86019 -122.19002   8.4035  1.262362e+09     4    39   

   p_picks  s_picks    rms  nsphz         gap algorithm  
0        2        5  1.081    5.0  243.895048     genie  
1       25       30  0.985   30.0  197.622556     genie  
2       10       18  0.784   18.0  179.587425     genie  
3       10       10  0.465   10.0  153.822127     genie  
4       20       19  0.657   19.0  238.225383     genie  


In [34]:
# Merge arrival_df with assoc_df on 'arid'
merged = arrival_df.merge(assoc_df, on='arid', how='left', suffixes=('', '_assoc'))

# Merge the result with origin_df on 'orid'
merged = merged.merge(origin_df, on='orid', how='left', suffixes=('', '_origin'))

# Drop duplicated columns (columns ending with '_assoc' or '_origin')
cols_to_drop = [col for col in merged.columns if col.endswith('_assoc') or col.endswith('_origin') or col == 'Unnamed: 0']
merged = merged.drop(columns=cols_to_drop, errors='ignore')

# Show the first few rows
assoc_df = merged.copy()
print(assoc_df.head())
assoc_df = assoc_df.to_csv('/wd1/hbito_data/data/datasets_all_regions/arrival_assoc_origin_2010_2015_reloc_cog_ver3_cc.csv')

       sta     pick_time  arid iphase  prob  orid phase  timeres  slatitude  \
0  UW.PCMD  1.262305e+09     0      P  0.68     0     P    0.049  46.888962   
1   UW.RVW  1.262305e+09     1      P  0.68     0     P    1.264  46.149750   
2  UW.PCMD  1.262305e+09     2      S  0.68     0     S   -0.243  46.888962   
3   UW.GNW  1.262305e+09     3      S  0.68     0     S    2.402  47.564130   
4  PB.B013  1.262305e+09     4      S  0.68     0     S   -0.651  47.813000   

   slongitude  ...        lon   depth         otime  nass  p_picks  s_picks  \
0 -122.301483  ... -122.09098  60.147  1.262305e+09     7        2        5   
1 -122.742996  ... -122.09098  60.147  1.262305e+09     7        2        5   
2 -122.301483  ... -122.09098  60.147  1.262305e+09     7        2        5   
3 -122.824980  ... -122.09098  60.147  1.262305e+09     7        2        5   
4 -122.910797  ... -122.09098  60.147  1.262305e+09     7        2        5   

     rms  nsphz         gap  algorithm  
0  1.081 

### Match with the catalog by Morton-2023

In [38]:
df_origin = pd.read_csv('/wd1/hbito_data/data/datasets_all_regions/origin_2010_2015_reloc_cog_ver3_cc.csv', index_col=0)
df_origin = df_origin
df_origin

,lat,lon,depth,time,orid,nass,p_picks,s_picks,rms,nsphz,gap,algorithm
0,47.13396,-122.09098,60.1470,1.262305e+09,0,7,2,5,1.081,5.0,243.895048,genie
7,48.17742,-121.83289,6.1630,1.262305e+09,1,55,25,30,0.985,30.0,197.622556,genie
62,47.85353,-122.12435,18.2960,1.262330e+09,2,28,10,18,0.784,18.0,179.587425,genie
90,47.97665,-122.90617,20.9100,1.262336e+09,3,20,10,10,0.465,10.0,153.822127,genie
110,45.86019,-122.19002,8.4035,1.262362e+09,4,39,20,19,0.657,19.0,238.225383,genie
...,...,...,...,...,...,...,...,...,...,...,...,...
1004270,40.59877,-124.40572,20.4825,1.435097e+09,63882,23,9,14,1.015,14.0,77.202555,genie
1004293,40.62611,-127.11445,18.5140,1.435100e+09,63883,12,1,11,0.744,11.0,221.215338,genie
1004305,49.76714,-124.52950,3.6850,1.435101e+09,63884,13,7,6,0.894,6.0,205.239215,genie
1004318,40.66547,-125.33902,3.0525,1.435101e+09,63885,8,3,5,0.982,5.0,141.141631,genie


In [39]:
events_morton = pd.read_csv('/wd1/hbito_data/data/ds01.csv')
# Convert the TSTRING to datetime
events_morton['datetime'] = pd.to_datetime(events_morton['TSTRING'], format='%Y%m%d%H%M%S', utc=True)
# Get the events in the Morton catalog 
# t1 = pd.Timestamp('2011-1-1 00:00:00.000000+0000', tz='UTC')
# t2 = pd.Timestamp('2015-12-31 23:59:59.999999+0000', tz='UTC')
events_morton['id_Morton'] = events_morton.index
# events_morton= events_morton.loc[(events_morton['datetime'] > t1) & (events_morton['datetime'] < t2) ]

events_morton.head()

,CI YEAR,TSTRING,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,LAT,LON,...,tt RMS,ERH,ERZ,STRIKE,DIP,RAKE,PLATE DESIGNATION,TEMPLATE EVENT?,datetime,id_Morton
0,1,2.011073e+13,2011,7,26,1,2,7.37,47.3217,-123.2708,...,0.19,0.8,1.2,NaN,NaN,NaN,Interface,Catalog,2011-07-26 01:02:07+00:00,0
1,1,2.011073e+13,2011,7,26,1,2,7.72,44.2888,-124.3340,...,0.06,13.1,3.2,NaN,NaN,NaN,Upper Plate,NaN,2011-07-26 01:02:07+00:00,1
2,1,2.011073e+13,2011,7,26,1,2,8.56,44.3017,-124.3180,...,0.50,35.4,22.2,NaN,NaN,NaN,Upper Plate,NaN,2011-07-26 01:02:08+00:00,2
3,1,2.011073e+13,2011,7,26,7,31,2.17,48.2635,-124.9298,...,0.77,3.5,6.4,NaN,NaN,NaN,Upper Plate,NaN,2011-07-26 07:31:02+00:00,3
4,1,2.011073e+13,2011,7,26,9,50,27.63,48.3032,-124.9157,...,0.94,4.0,6.9,NaN,NaN,NaN,Upper Plate,T,2011-07-26 09:50:27+00:00,4


In [40]:
# Create a list to store the results
closest_morton_events = []

# Loop over each origin in df_origin
for _, origin_row in tqdm(df_origin.iterrows(), total=df_origin.shape[0]):
    origin_lat = origin_row['lat']
    origin_lon = origin_row['lon']
    origin_time = origin_row['time']

    # Calculate the distance and time difference for each Morton event
    events_morton['location_diff'] = events_morton.apply(
        lambda row: locations2degrees(origin_lat, origin_lon, row['LAT'], row['LON']), axis=1
    )
    events_morton['time_diff'] = abs(events_morton['datetime'].apply(lambda x: x.timestamp()) - origin_time)

    events_morton['NonDimDist'] = np.sqrt(
        (events_morton['location_diff'] / 25) ** 2 + (events_morton['time_diff'] / 120) ** 2)
    
    # Find the Morton event with the smallest location difference
    closest_event = events_morton.loc[events_morton['NonDimDist'].idxmin()]

    

    # Append the result
    closest_morton_events.append({
        'orid': origin_row['orid'],
        'lat': origin_row['lat'],
        'lon': origin_row['lon'],
        'depth': origin_row['depth'],
        'time': origin_row['time'],
        'nass': origin_row['nass'],
        'p_picks': origin_row['p_picks'],
        's_picks': origin_row['s_picks'],
        'rms': origin_row['rms'],
        'nsphz': origin_row['nsphz'],
        'gap': origin_row['gap'],
        'algorithm': origin_row['algorithm'],
        'id_Morton': closest_event['id_Morton'],
        'dist': closest_event['location_diff'],
        'dt': closest_event['time_diff'],
        'NonDimDist': closest_event['NonDimDist'],
    })

# Convert the results to a DataFrame
df_closest_morton_events = pd.DataFrame(closest_morton_events)
df_closest_morton_events

100%|██████████| 63887/63887 [3:37:49<00:00,  4.89it/s]  


,orid,lat,lon,depth,time,nass,p_picks,s_picks,rms,nsphz,gap,algorithm,id_Morton,dist,dt,NonDimDist
0,0,47.13396,-122.09098,60.1470,1.262305e+09,7,2,5,1.081,5.0,243.895048,genie,0,0.822891,4.933721e+07,411143.423300
1,1,48.17742,-121.83289,6.1630,1.262305e+09,55,25,30,0.985,30.0,197.622556,genie,0,1.291074,4.933712e+07,411142.647142
2,2,47.85353,-122.12435,18.2960,1.262330e+09,28,10,18,0.784,18.0,179.587425,genie,0,0.938460,4.931184e+07,410932.026358
3,3,47.97665,-122.90617,20.9100,1.262336e+09,20,10,10,0.465,10.0,153.822127,genie,0,0.699496,4.930621e+07,410885.090033
4,4,45.86019,-122.19002,8.4035,1.262362e+09,39,20,19,0.657,19.0,238.225383,genie,0,1.639359,4.927976e+07,410664.692283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63882,63882,40.59877,-124.40572,20.4825,1.435097e+09,23,9,14,1.015,14.0,77.202555,genie,4408,0.101096,7.917828e+03,65.981900
63883,63883,40.62611,-127.11445,18.5140,1.435100e+09,12,1,11,0.744,11.0,221.215338,genie,4408,2.149445,9.994498e+03,83.287528
63884,63884,49.76714,-124.52950,3.6850,1.435101e+09,13,7,6,0.894,6.0,205.239215,genie,4408,9.213035,1.107416e+04,92.285427
63885,63885,40.66547,-125.33902,3.0525,1.435101e+09,8,3,5,0.982,5.0,141.141631,genie,4408,0.807424,1.158678e+04,96.556489


In [ ]:
df_closest_morton_events.to_csv('/wd1/hbito_data/data/datasets_all_regions/origin_2010_2015_reloc_cog_morton_ver3_cc.csv')

In [ ]:
test = pd.read_csv('/wd1/hbito_data/data/datasets_all_regions/origin_2010_2015_reloc_cog_morton_ver3_cc.csv', index_col=0)
test

,lat,lon,depth,time,orid,nass,p_picks,s_picks,rms,nsphz,gap,algorithm,id_Morton,dist,dt,NonDimDist
0,47.22533,-122.16895,56.111,1.262305e+09,0,7,2,5,1.081,5.0,235.831208,genie,0,0.753784,4.933721e+07,411143.414481
1,48.19518,-121.77276,3.820,1.262305e+09,1,55,25,30,0.985,30.0,201.698107,genie,0,1.333054,4.933712e+07,411142.646872
2,47.86208,-122.09903,17.799,1.262330e+09,2,28,10,18,0.784,18.0,181.023074,genie,0,0.957326,4.931184e+07,410932.027590
3,47.96435,-122.91906,21.286,1.262336e+09,3,20,10,10,0.465,10.0,150.528010,genie,0,0.684951,4.930621e+07,410885.088574
4,45.87262,-122.19180,9.822,1.262362e+09,4,39,20,19,0.657,19.0,237.324295,genie,0,1.627692,4.927976e+07,410664.693011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63882,40.59439,-124.42438,19.741,1.435097e+09,63882,23,9,14,1.015,14.0,73.959960,genie,4408,0.112501,7.919735e+03,65.997794
63883,40.54636,-127.10076,26.165,1.435100e+09,63883,12,1,11,0.744,11.0,223.080984,genie,4408,2.139193,9.995899e+03,83.299204
63884,49.74167,-124.58578,5.274,1.435101e+09,63884,13,7,6,0.894,6.0,197.904075,genie,4408,9.188393,1.107364e+04,92.281049
63885,40.64363,-125.36531,1.891,1.435101e+09,63885,8,3,5,0.982,5.0,145.748415,genie,4408,0.824720,1.158579e+04,96.548245


In [ ]:
pd.to_datetime(df_closest_morton_events['dt'][0].copy(), utc=True)

In [ ]:
import datetime

# Convert the time difference in seconds to a human-readable format
df_closest_morton_events['dt_readable'] = df_closest_morton_events['dt'].apply(
    lambda x: str(datetime.timedelta(seconds=x))
)

df_closest_morton_events

In [ ]:
df_closest_morton_events_temp = df_closest_morton_events[['orid','id_Morton','dist','dt','NonDimDist']].copy()
df_closest_morton_events_temp

In [ ]:
df_closest_morton_events = df_origin.merge(df_closest_morton_events_temp, on='orid', how='left')
df_closest_morton_events 

In [ ]:
df_closest_morton_events['dt_readable'] = df_closest_morton_events['dt'].apply(
    lambda x: str(datetime.timedelta(seconds=x))
)

df_closest_morton_events

In [ ]:
df_closest_morton_events['dt_readable'] = df_closest_morton_events['dt'].apply(
    lambda x: str(datetime.timedelta(seconds=x))
)

df_closest_morton_events

In [ ]:
str(datetime.timedelta(seconds=df_closest_morton_events.loc[df_closest_morton_events.orid==13224]['dt'].copy()))

In [ ]:
df_closest_morton_events.loc[df_closest_morton_events.orid==13224]['dt']

In [ ]:
matched_events = pd.read_csv('../data/datasets_all_regions/matched_events_with_morton_mycatalog_reloc_cog_ver3.csv', index_col=0)
matched_events

In [ ]:
test = pd.read_csv('')